In [2]:
import numpy as np 
import pandas as pd 
import warnings
warnings.filterwarnings("ignore")

#import src
import Cleanup
import Transformation
import FileProcessing

In [3]:
#define the objects to be called later. 
cleanup=Cleanup.Cleanup()
transform=Transformation.Transformation()
fileProcessing=FileProcessing.FileProcessing()

Read the File from Data folder and remove all duplicates. 

In [4]:
file_name='Next_Generation_Simulation__NGSIM__Vehicle_Trajectories_and_Supporting_Data'
ngsim=fileProcessing.read_input(file_name)

original File path: c:\Users\StudentAccount\Python\Capstone\gitcodelocation\DataDrivenCarFollowing\datadrivencarfollowing-v1\scripts
Data File path: c:\Users\StudentAccount\Python\Capstone\gitcodelocation\DataDrivenCarFollowing\datadrivencarfollowing-v1\data


Remove any Duplicate records from the original File. 

In [5]:
cleanup.remove_dups(ngsim)

704000 duplicate values have been removed


,Vehicle_ID,Frame_ID,Total_Frames,Global_Time,Local_X,Local_Y,Global_X,Global_Y,v_length,v_Width,...,D_Zone,Int_ID,Section_ID,Direction,Movement,Preceding,Following,Space_Headway,Time_Headway,Location
0,515,2330,1123,1118848075000,30.034,188.062,6451203.729,1873252.549,13.0,6.9,...,NaN,NaN,NaN,NaN,NaN,500,523,119.10,5.11,us-101
2,2224,6548,1902,1113437421700,41.429,472.901,6042814.264,2133542.012,14.3,6.9,...,NaN,NaN,NaN,NaN,NaN,2208,2211,53.34,2.01,i-80
3,2127,6459,567,1118847624800,19.632,1775.614,6452425.122,1872172.475,13.5,6.9,...,NaN,NaN,NaN,NaN,NaN,2124,2132,48.92,1.30,us-101
4,1033,4827,592,1118848324700,6.202,1701.144,6452347.673,1872258.452,13.5,4.4,...,NaN,NaN,NaN,NaN,NaN,1029,1040,38.81,0.92,us-101
6,1890,9157,628,1118849672700,53.514,817.521,6451655.238,1872800.663,24.0,8.5,...,NaN,NaN,NaN,NaN,NaN,1882,1897,102.65,2.27,us-101
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11850521,1355,5622,938,1113438127100,18.275,1464.336,6042648.149,2134520.896,12.3,6.8,...,NaN,NaN,NaN,NaN,NaN,1351,1361,26.91,15.92,i-80
11850522,1474,5688,401,1113438133700,5.509,560.412,6042767.840,2133624.549,17.8,7.9,...,NaN,NaN,NaN,NaN,NaN,1468,1480,74.49,1.73,i-80
11850523,398,2368,654,1113437801700,18.903,635.841,6042771.741,2133701.076,15.2,8.5,...,NaN,NaN,NaN,NaN,NaN,385,406,90.95,3.77,i-80
11850524,599,875,577,1113436854400,77.094,621.358,6042831.282,2133693.854,15.3,6.4,...,NaN,NaN,NaN,NaN,NaN,0,611,0.00,0.00,i-80


Feet to Metres:

Below Four parametrs consider distance in Feet and they are converted to metre. Speed to Metre/ second from Feet/second and acceleration from feet/second Square to Metre/Second Square. 

Drop the columns we wont use in the model.
Add Relative time which is the time from the first timframe in the dataset. This is calculated by using the minimum value ofGlobal time(absolute time from 1970)

In [6]:
ngsim=transform.convert_feet_to_metre(ngsim)

In [7]:
ngsim.shape

(11146526, 25)

In [8]:
ngsim=transform.drop_not_required_columns(ngsim)

Create Placeholders for the user defined columns we will be populating later, to be used in the model. 
Map the Vehicle classes to their english equivalent. 
Filter the original file into US-101 and I-80 to perform and populate the required fields later. These files contain same values of Vehicle ID used for multiple vehicles based on the highway. 



In [9]:
#filtered_ngsim= ngsim[((ngsim['Location'] == 'us-101') | (ngsim['Location'] == 'i-80')) & (ngsim['Preceding']>0) & (ngsim['Following']>0) ]
ngsim=transform.create_column_placeholders(ngsim)

Split data into US-101 and I-80 specific dataframes.

In [10]:
filtered_ngsim_U,filtered_ngsim_I = transform.bifurcate_highways(ngsim)

Create Sets to use and create the L-F Pairs. 

In [11]:
v_Class_M_U,v_Class_C_U,v_Class_HV_U = transform.class_vehicle_sets(filtered_ngsim_U)
v_Class_M_I,v_Class_C_I,v_Class_HV_I = transform.class_vehicle_sets(filtered_ngsim_I)


Add the Preceding Vehicle Class to the new previous class column

In [12]:
filtered_ngsim_U=transform.preceding_vehicle_class(filtered_ngsim_U,v_Class_M_U,v_Class_C_U,v_Class_HV_U)
filtered_ngsim_I=transform.preceding_vehicle_class(filtered_ngsim_I,v_Class_M_I,v_Class_C_I,v_Class_HV_I)

Map the Vehicle Length of the previous vehicle into the row. 

In [13]:
filtered_ngsim_U=transform.preceding_vehicle_length(filtered_ngsim_U)
filtered_ngsim_I=transform.preceding_vehicle_length(filtered_ngsim_I)

Convert the Front to Front Space Headway to Back Bumper (preceding Vehicle) to front Bumper(of Following ) Headway. This will give the actual distance between the vehicle ignoring the vehicle length.
Find the time it would take for the Following vehicle to reach the rear Bumper of the Preceding Vehicle. 

In [14]:
filtered_ngsim_U= transform.front_to_front_bumper_details_changed_to_rear_to_front_bumper_details(filtered_ngsim_U)
filtered_ngsim_I= transform.front_to_front_bumper_details_changed_to_rear_to_front_bumper_details(filtered_ngsim_I)

In [15]:
filtered_ngsim_U.columns

Index(['Vehicle_ID', 'Frame_ID', 'Global_Time', 'Local_X', 'Local_Y',
       'v_length', 'v_Class', 'v_Vel', 'v_Acc', 'Lane_ID', 'Preceding',
       'Following', 'Space_Headway', 'Time_Headway', 'Location',
       'Preceding_Vehicle_Class', 'Rear_to_Front_Space_Headway',
       'Front_To_Rear_Time_Headway', 'Velocity Difference_Following-Preceding',
       'Acceleration Difference_Following-Preceding', 'L-F_Pair',
       'v_Class_Name', 'Vehicle_combination', 'preceding_vehicle_length'],
      dtype='object')

Vehicle Combination of the Preceding and the Following Vehicle. 

In [16]:
filtered_ngsim_U=transform.map_preceding_vehicle_details(filtered_ngsim_U)
filtered_ngsim_I=transform.map_preceding_vehicle_details(filtered_ngsim_I)

us-101::621 cars dont change lanes
us-101::2226 cars Change lanes
i-80::817 cars dont change lanes
i-80::2184 cars Change lanes


In [17]:
filtered_ngsim_U=transform.map_pairs(filtered_ngsim_U)
filtered_ngsim_I=transform.map_pairs(filtered_ngsim_I)

In [18]:
filtered_ngsim= fileProcessing.merge_files(filtered_ngsim_U , filtered_ngsim_I)

verify_pairs = filtered_ngsim[['L-F_Pair', 'Location']]

verify_pairs.drop_duplicates(inplace=True)
verify_pairs = verify_pairs.groupby(
    ['L-F_Pair'], as_index=False).count()
# print(lane_verify.shape)
duplicate_pairs = set(
    verify_pairs[verify_pairs["Location"] > 1]['L-F_Pair'])


 Merged Record Count:8665320, df1:4098933, df2:4566387


In [19]:
print(filtered_ngsim_U[filtered_ngsim_U['L-F_Pair'].isin(duplicate_pairs)]['Vehicle_ID'].count())
print(filtered_ngsim_I[filtered_ngsim_I['L-F_Pair'].isin(duplicate_pairs)]['Vehicle_ID'].count())

853964
907929


In [20]:
#len(duplicate_pairs)

In [21]:
#len(filtered_ngsim['L-F_Pair'].unique())

In [22]:
filtered_ngsim_U=cleanup.filter_records_for_model(filtered_ngsim_U)
filtered_ngsim_I=cleanup.filter_records_for_model(filtered_ngsim_I)

filtered_ngsim_U.drop(columns=["total_pair_duration",'pair_Time_Duration'],axis=1,inplace=True)
filtered_ngsim_I.drop(columns=["total_pair_duration",'pair_Time_Duration'],axis=1,inplace=True)
filtered_ngsim_U=transform.map_pairs(filtered_ngsim_U)
filtered_ngsim_I=transform.map_pairs(filtered_ngsim_I)

dataset before Row Removal(4098933, 33)
us-101: 157194 vehicles first and Last 5 seconds removed from Lane 1 due to lane changing
us-101: 10594 vehicles first and Last 5 seconds removed from Lane 2 due to lane changing
us-101: 8027 vehicles first and Last 5 seconds removed from Lane 3 due to lane changing
us-101: 8961 vehicles first and Last 5 seconds removed from Lane 4 due to lane changing
us-101: 9343 vehicles first and Last 5 seconds removed from Lane 5 due to lane changing
us-101: 3589 vehicles first and Last 5 seconds removed from Lane 6 due to lane changing
us-101: 2755 vehicles first and Last 5 seconds removed from Lane 7 due to lane changing
us-101: 1027 vehicles first and Last 5 seconds removed from Lane 8 due to lane changing
us-101: 271468 have multiple lengths and classes for same Vehicle ID
us-101: 780619 have total car following less than 30 seconds
us-101: 266321 have time Headway less than 5 seconds
us-101: 18010 have Lane ID as 7 or 8 which are the Ramps
us-101: 11363


1. Velocity difference of the Lead minus Preceding Vehicle.
2. Acceleration difference of the Lead minus Preceding Vehicle.
3. Find the Pair details of the Lead_following 
4. Find the cumulative time the Pair runs within the highway as a pair. 

In [23]:
filtered_ngsim= fileProcessing.merge_files(filtered_ngsim_U , filtered_ngsim_I)
print(filtered_ngsim_U[filtered_ngsim_U['L-F_Pair'].isin(duplicate_pairs)]['Vehicle_ID'].count())
print(filtered_ngsim_I[filtered_ngsim_I['L-F_Pair'].isin(duplicate_pairs)]['Vehicle_ID'].count())
#print(filtered_ngsim_U[filtered_ngsim_U['L-F_Pair'].isin(duplicate_pairs)]['Vehicle_ID'].count())
#print(filtered_ngsim_I[filtered_ngsim_I['L-F_Pair'].isin(duplicate_pairs)]['Vehicle_ID'].count())

#fileProcessing.export_file(filtered_ngsim,'Cleaned_NGSIM_Data')

 Merged Record Count:6097192, df1:2962617, df2:3134575
550234
560111


In [24]:
filtered_ngsim.columns

Index(['Vehicle_ID', 'Frame_ID', 'Global_Time', 'Local_X', 'Local_Y',
       'v_length', 'v_Class', 'v_Vel', 'v_Acc', 'Lane_ID', 'Preceding',
       'Following', 'Space_Headway', 'Time_Headway', 'Location',
       'Preceding_Vehicle_Class', 'Rear_to_Front_Space_Headway',
       'Front_To_Rear_Time_Headway', 'Velocity Difference_Following-Preceding',
       'Acceleration Difference_Following-Preceding', 'L-F_Pair',
       'v_Class_Name', 'Vehicle_combination', 'preceding_vehicle_length',
       'lane_changes', 'Prec_Vehicle_ID', 'preceding_Vehicle_Velocity',
       'preceding_Vehicle_Acceleration', 'preceding_car_lane_changes',
       'preceding_Local_Y', 'preceding_v_Class', 'pair_Time_Duration',
       'total_pair_duration'],
      dtype='object')

In [25]:
print(f"{filtered_ngsim[(filtered_ngsim['preceding_car_lane_changes'] == True) ]['Vehicle_ID'].unique().size} cars Change lanes")
print(f"{filtered_ngsim[(filtered_ngsim['preceding_car_lane_changes'] == False) ]['Vehicle_ID'].unique().size} cars DonT")

2746 cars Change lanes
1280 cars DonT


In [26]:
#filtered_ngsim[((filtered_ngsim['Vehicle_ID'] <= 100 )  ) & ((filtered_ngsim['Preceding'] <= 100 )  )].to_csv('classVERIFYFILE.CSV', index = False)

In [27]:
filtered_ngsim

,Vehicle_ID,Frame_ID,Global_Time,Local_X,Local_Y,v_length,v_Class,v_Vel,v_Acc,Lane_ID,...,preceding_vehicle_length,lane_changes,Prec_Vehicle_ID,preceding_Vehicle_Velocity,preceding_Vehicle_Acceleration,preceding_car_lane_changes,preceding_Local_Y,preceding_v_Class,pair_Time_Duration,total_pair_duration
0,8,29,1118846981800,11.236757,11.727485,4.72440,2,13.716000,0.000000,4,...,4.67360,True,5,11.783568,-3.322320,True,37.322150,2.0,0.0,65.8
1,8,30,1118846981900,11.230051,13.099085,4.72440,2,13.716000,0.000000,4,...,4.67360,True,5,11.430000,-3.413760,True,38.482219,2.0,0.1,65.8
2,8,31,1118846982000,11.222736,14.470685,4.72440,2,13.716000,0.000000,4,...,4.67360,True,5,11.076432,-3.319272,True,39.607846,2.0,0.2,65.8
3,8,32,1118846982100,11.215726,15.842285,4.72440,2,13.716000,0.000000,4,...,4.67360,True,5,10.820400,-1.886712,True,40.689581,2.0,0.3,65.8
4,8,33,1118846982200,11.208715,17.213885,4.72440,2,13.716000,0.000000,4,...,4.67360,True,5,10.689336,-0.569976,True,41.753333,2.0,0.4,65.8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3134570,2485,11650,1113438729900,16.713403,489.215278,4.20624,2,1.069848,0.332232,5,...,4.52628,True,2460,1.947672,0.826008,True,498.871342,2.0,128.0,128.4
3134571,2485,11651,1113438730000,16.714013,489.316776,4.20624,2,1.170432,1.487424,5,...,4.52628,True,2460,1.953768,0.015240,True,499.073424,2.0,128.1,128.4
3134572,2485,11652,1113438730100,16.714927,489.431990,4.20624,2,1.383792,2.868168,5,...,4.52628,True,2460,1.953768,0.000000,True,499.309949,2.0,128.2,128.4
3134573,2485,11653,1113438730200,16.717366,489.581342,4.20624,2,1.694688,3.413760,5,...,4.52628,True,2460,1.953768,0.000000,True,499.462349,2.0,128.3,128.4


In [28]:
filtered_ngsim['L-F_Pair'].unique().shape

(9133,)

In [29]:
filtered_ngsim[((filtered_ngsim['Space_Headway'] == 0 ) & (filtered_ngsim['Preceding'] > 0 )) ]


,Vehicle_ID,Frame_ID,Global_Time,Local_X,Local_Y,v_length,v_Class,v_Vel,v_Acc,Lane_ID,...,preceding_vehicle_length,lane_changes,Prec_Vehicle_ID,preceding_Vehicle_Velocity,preceding_Vehicle_Acceleration,preceding_car_lane_changes,preceding_Local_Y,preceding_v_Class,pair_Time_Duration,total_pair_duration
2141580,572,3292,1113437894100,5.464454,452.776438,4.35864,2,0.0,0.0,2,...,4.81584,True,566,0.749808,0.103632,True,452.777657,2.0,23.9,34.2


In [30]:
filtered_ngsim[((filtered_ngsim['Time_Headway'] <= 5 ) )]['L-F_Pair'].unique().shape


(9122,)

In [32]:
filtered_ngsim['L-F_Pair'].unique().shape

(9133,)

In [31]:
filtered_ngsim.columns

Index(['Vehicle_ID', 'Frame_ID', 'Global_Time', 'Local_X', 'Local_Y',
       'v_length', 'v_Class', 'v_Vel', 'v_Acc', 'Lane_ID', 'Preceding',
       'Following', 'Space_Headway', 'Time_Headway', 'Location',
       'Preceding_Vehicle_Class', 'Rear_to_Front_Space_Headway',
       'Front_To_Rear_Time_Headway', 'Velocity Difference_Following-Preceding',
       'Acceleration Difference_Following-Preceding', 'L-F_Pair',
       'v_Class_Name', 'Vehicle_combination', 'preceding_vehicle_length',
       'lane_changes', 'Prec_Vehicle_ID', 'preceding_Vehicle_Velocity',
       'preceding_Vehicle_Acceleration', 'preceding_car_lane_changes',
       'preceding_Local_Y', 'preceding_v_Class', 'pair_Time_Duration',
       'total_pair_duration'],
      dtype='object')